# Testing notebook

In [1]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [2]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [3]:
from computions import *

### get_describe_nominal function test

Basic situation

In [4]:
get_describe_nominal(test_frame.iloc[:,1], Y)

object_variable


,count,2.0,2.0%,1.0,1.0%,0.0,0.0%
object_variable,,,,,,,
2.0,112,0.0,0.000000,34,30.357143,78.0,69.642857
8.0,112,32.0,28.571429,80,71.428571,0.0,0.000000
7.0,108,8.0,7.407407,100,92.592593,0.0,0.000000
5.0,105,4.0,3.809524,89,84.761905,12.0,11.428571
6.0,100,3.0,3.000000,95,95.000000,2.0,2.000000
1.0,96,0.0,0.000000,11,11.458333,85.0,88.541667
4.0,95,0.0,0.000000,71,74.736842,24.0,25.263158
9.0,95,53.0,55.789474,42,44.210526,0.0,0.000000
3.0,89,0.0,0.000000,47,52.808989,42.0,47.191011


Data frame with emptys

In [5]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))

object_variable
Is all obs. are in table -  False
object_variable
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [6]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{2.0: 0.773740053050398, 1.0: 0.5282017362546507, 0.0: 0.3125892857142857}
{'category 2': 0.773740053050398, 'category 1': 0.5282017362546507, 'cetegory 0': 0.3125892857142857}
{'category 2': 0.7712586195902678, 'category 1': 0.5283229182805694, 'cetegory 0': 0.3116195450000259}
{1.0: 0.6874107142857143, 0.0: 0.3125892857142857}


AUC nominal

In [7]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys')
print(get_AUC_numeric(na_containts_frame['object_variable'], Y_strs))

object_variable
object_variable
{'category 2': 0.9169982316534041, 'category 1': 0.8360789582472096, 'cetegory 0': 0.9380109126984127}
AUC bynary
object_variable
{1.0: 0.9380109126984126, 0.0: 0.9380109126984127}
AUC with emptys
{'category 2': 0.918381286941006, 'category 1': 0.6728156675884075, 'cetegory 0': 0.06245025188593494}


Get publish AUC for any predictor type

In [8]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)

with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

object_variable
nominal predictor without table
{2.0: {'AUC': 0.9169982316534041, 'rel_type': 1, 'GINI': 0.8339964633068082}, 1.0: {'AUC': 0.8360789582472096, 'rel_type': 1, 'GINI': 0.6721579164944191}, 0.0: {'AUC': 0.9380109126984127, 'rel_type': 1, 'GINI': 0.8760218253968255}}
moninal predictor with table
{'category 2': {'AUC': 0.9169982316534041, 'rel_type': 1, 'GINI': 0.8339964633068082}, 'category 1': {'AUC': 0.8360789582472096, 'rel_type': 1, 'GINI': 0.6721579164944191}, 'cetegory 0': {'AUC': 0.9380109126984127, 'rel_type': 1, 'GINI': 0.8760218253968255}}
numeric predictor
{2.0: {'AUC': 0.773740053050398, 'rel_type': 1, 'GINI': 0.5474801061007959}, 1.0: {'AUC': 0.5282017362546507, 'rel_type': 1, 'GINI': 0.056403472509301444}, 0.0: {'AUC': 0.6874107142857143, 'rel_type': -1, 'GINI': 0.37482142857142864}}


Decoding AUC info as DataFrame

In [9]:
numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
AUC_info_to_DataFrame(numeric_var,'numeric_variable')

2.0                         1.0                     \
                      AUC rel_type     GINI       AUC rel_type      GINI   
numeric_variable  0.77374        1  0.54748  0.528202        1  0.056403   

                       0.0                     
                       AUC rel_type      GINI  
numeric_variable  0.687411       -1  0.374821

## Writing to excel

Adding a new sheet

In [10]:
xl_writer = pd.ExcelWriter("test_result/sheet_creation.xlsx",engine='xlsxwriter')

for i in range(20):
    add_sheet(xl_writer, 'some name')
    add_sheet(xl_writer, 'some very very very long long name')
xl_writer.close()

Adding a info about different predictors

In [11]:
xl_writer = pd.ExcelWriter("test_result/write_to_book.xlsx",engine='xlsxwriter')

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

object_variable


Let's show final table

In [ ]:
my_cpp.result_DF

0.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.717273       -1  0.434546   
object_variable                                0.922994        1  0.845989   
column with soo000oo000oo000oo000oo long name  0.530123       -1  0.060246   

                                                    1.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.529318        1  0.058637   
object_variable                                0.814341        1  0.628681   
column with soo000oo000oo000oo000oo long name  0.543274        1  0.086548   

                                                    2.0                     
                                                    AUC rel_type      GINI  
numeric_variable                                 0.7424        1    0.4848  
object_variable                                0.923709        1  0.847418  
column with soo000oo000oo000oo000oo long name  0.533278       -1  0.066556

In [25]:
my_cpp.write_to_book(xl_writer)

xl_writer.close()